## Modelando con toda la data y mejores parametros

In [6]:
import pandas as pd
import json
from pathlib import Path
import mlflow

# Importar nuestras funciones personalizadas
from package_ml.modeling.train import train_final_model
from package_ml.modeling.predict import make_predictions, evaluate_model_on_test_set

In [7]:
# --- Definición de Rutas y Constantes ---
DATA_DIR = Path('../data/interim')
MODELS_DIR = Path('../models')
REPORTS_DIR = Path('../reports')

TARGET = "Calories"

In [8]:
# Cargar datos de entrenamiento
df_train = pd.read_parquet(DATA_DIR / 'train_fe.parquet')

# Cargar la lista de características seleccionadas
with open(MODELS_DIR / 'final_features.json', 'r') as f:
    final_features = json.load(f)

# Cargar los mejores hiperparámetros
with open(MODELS_DIR / 'best_lgbm_params.json', 'r') as f:
    best_params = json.load(f)

# Preparar datos de entrenamiento
X_train = df_train[final_features]
y_train = df_train[TARGET]

In [9]:
# Definimos la ruta donde se guardará el modelo de producción
final_model_path = MODELS_DIR / 'final_lgbm_model.joblib'

# Llamar a la función de entrenamiento
train_final_model(X_train, y_train, best_params, final_model_path)

--- Entrenando el modelo final con todos los datos de entrenamiento... ---
Modelo final entrenado y guardado en: ../models/final_lgbm_model.joblib


## Realizando la prediccion sobre data no vista

In [10]:
# 1. Cargar los datos de prueba
df_test = pd.read_parquet(DATA_DIR / 'test_fe.parquet')

# 2. Preparar datos de prueba
X_test = df_test[final_features]  # Asegurarse de usar LAS MISMAS features
y_test = df_test[TARGET]

# 3. Realizar predicciones
predictions = make_predictions(X_test, final_model_path)

# 4. Evaluar las predicciones y obtener las métricas
test_metrics = evaluate_model_on_test_set(y_test, predictions)

--- Cargando modelo desde ../models/final_lgbm_model.joblib para realizar predicciones... ---
--- Evaluando el rendimiento del modelo en el conjunto de prueba... ---
Resultados en datos no vistos: {'rmse_test': 3.4070282641499783, 'mae_test': 2.1193435668310907, 'r2_test': 0.9969150808977788}


## Conclusion de la evaluación final

In [1]:
El modelo final se evaluó en un conjunto de datos de prueba completamente nuevo para verificar su capacidad de generalización.

- **RMSE en Validación Cruzada (Optuna, 200k datos):** `3.66`
- **RMSE en el Conjunto de Prueba:** `3.40`

**Análisis de Resultados:**
El rendimiento en el conjunto de prueba es excelente y consistente con el obtenido durante la validación cruzada.

El RMSE final de **3.40 calorías** representa una estimación fiable del error esperado del modelo en producción. El modelo se considera robusto y listo para su despliegue.

SyntaxError: invalid decimal literal (1887218076.py, line 3)